In [1]:
import shutil
import os
directory = "/content/dataset"
if any(os.listdir(directory)):
  print('dataset exists')
else:
  !unzip /content/SETA_dataset -d /content/

Archive:  /content/SETA_dataset.zip
  inflating: /content/SETA_dataset/.DS_Store  
  inflating: /content/SETA_dataset/video_X_test.pkl  
  inflating: /content/SETA_dataset/video_X_train.pkl  
  inflating: /content/SETA_dataset/video_X_valid.pkl  
  inflating: /content/SETA_dataset/video_y_test.pkl  
  inflating: /content/SETA_dataset/video_y_train.pkl  
  inflating: /content/SETA_dataset/video_y_valid.pkl  
  inflating: /content/SETA_dataset/ziping.py  


In [1]:
import os
import pickle
import numpy as np
import matplotlib.pyplot as plt
import itertools
import tensorflow as tf
import pandas as pd

import tensorflow
from tensorflow.keras import backend as K
from utility import LoadDataIot
from Model_cnn_best import DCNet   #DCNet_Dropouts
import random
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler
from tensorflow.keras.optimizers import Adamax
#import tensorflow_addons as tf

from sklearn.utils import shuffle
#from tensorflow_addons.optimizers import extend_with_decoupled_weight_decay
from tensorflow.keras.models import Model
from tensorflow.python.ops.numpy_ops import np_config
from sklearn.metrics import accuracy_score
np_config.enable_numpy_behavior()


In [5]:
!python dataset/Class_filter_SETA.py

2023-10-08 18:02:27.983716: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
{'closed': [0, 1, 2, 3, 4, 5, 6, 7], 'open': [8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]}
Traceback (most recent call last):
  File "/content/dataset/Class_filter_SETA.py", line 100, in <module>
    X_train = pickle.load(f)
_pickle.UnpicklingError: pickle data was truncated


In [8]:
# Training the DF model
NB_EPOCH = 450   # Number of training epoch
print("Number of Epoch: ", NB_EPOCH)
BATCH_SIZE = 64 # Batch size
VERBOSE = 2 # Output display mode
LENGTH = 500 # Packet sequence length
OPTIMIZER = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8) # Optimizer

NB_CLASSES = 8 # number of outputs = number of classes
INPUT_SHAPE = (LENGTH,1)

random.seed(0)

def representative_data_gen():
  for input_value in tensorflow.data.Dataset.from_tensor_slices(X_train_Rep).batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]


def Micro_F1(matrix):
    epsilon = 1e-8
    TP = 0
    FP = 0
    TN = 0

    for k in range(NB_CLASSES):
        TP += matrix[k][k]
        FP += (np.sum(Matrix, axis=0)[k] - matrix[k][k])
        TN += (np.sum(Matrix, axis=1)[k] - matrix[k][k])

    Micro_Prec = TP / (TP + FP)
    Micro_Rec = TP / (TP + TN)
    print("Micro Precision: ", Micro_Prec)
    print("Micro Recall: ", Micro_Rec)
    Micro_F1 = 2 * Micro_Prec * Micro_Rec / (Micro_Rec + Micro_Prec + epsilon)

    return Micro_F1


def New_F1_Score(Matrix):
    Column_sum = np.sum(Matrix, axis=0)
    Raw_sum = np.sum(Matrix, axis=1)

    Precision_Differences = []
    Recall_Differences = []
    for i in range(NB_CLASSES):
        Precision_Differences.append(np.abs(2 * Matrix[i][i] - Column_sum[i]))
        Recall_Differences.append(np.abs(2 * Matrix[i][i] - Raw_sum[i]))

    Precision_Differences = np.array(Precision_Differences)
    Precision_Differences_Per = Precision_Differences / np.sum(Precision_Differences)
    Recall_Differences = np.array(Recall_Differences)
    Recall_Differences_Per = Recall_Differences / np.sum(Recall_Differences)

    # print('Precision_Differences_Per',Precision_Differences_Per)
    # print('Recall_Differences_Per',Recall_Differences_Per)

    Precisions = np.zeros(NB_CLASSES)
    Recalls = np.zeros(NB_CLASSES)

    epsilon = 1e-8

    for k in range(len(Precisions)):
        Precisions[k] = (Matrix[k][k] / np.sum(Matrix, axis=0)[k])
    Precision = np.sum(np.array(Precisions) * Precision_Differences_Per)

    for k in range(len(Recalls)):
        Recalls[k] = (Matrix[k][k] / np.sum(Matrix, axis=1)[k])  # *Recall_Differences_Per[k]
    Recall = np.sum(np.array(Recalls) * Recall_Differences_Per)

    print('Precision: ', Precision)
    print('Recall: ', Recall)

    F1_Score = 2 * Precision * Recall / (Precision + Recall + epsilon)
    return F1_Score


def Macro_F1(Matrix):
    Precisions = np.zeros(NB_CLASSES)
    Recalls = np.zeros(NB_CLASSES)

    epsilon = 1e-8

    for k in range(len(Precisions)):
        Precisions[k] = Matrix[k][k] / np.sum(Matrix, axis=0)[k]
    # print(Precisions)

    Precision = np.average(Precisions)
    print("Precision:", Precision)

    for k in range(len(Recalls)):
        Recalls[k] = Matrix[k][k] / np.sum(Matrix, axis=1)[k]

    Recall = np.average(Recalls)
    print("Recall:", Recall)

    F1_Score = 2 * Precision * Recall / (Precision + Recall + epsilon)
    return F1_Score

# Data: shuffled and split between train and test sets
print ("Loading and preparing data for training, and evaluating the model")
X_train, y_train, X_test, y_test, X_valid, y_valid  = LoadDataIot()

# Convert data as float32 type
X_train = X_train.astype('float32')
X_valid = X_valid.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_valid = y_valid.astype('float32')
y_test = y_test.astype('float32')
# we need a [Length x 1] x n shape as input to the DF CNN (Tensorflow)
X_train = X_train[:, :,np.newaxis]
X_valid = X_valid[:, :,np.newaxis]
X_test = X_test[:, :,np.newaxis]


y_train = to_categorical(y_train, NB_CLASSES)
y_valid = to_categorical(y_valid, NB_CLASSES)
y_test = to_categorical(y_test, NB_CLASSES)\

# Building and training model
print("Building and training DF model")

model = DCNet.build(input_shape=INPUT_SHAPE, nb_classes=NB_CLASSES)

model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
    metrics=["accuracy"])

filepath = '/content/training_history/SETA.hdf5'

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

# callbacks_list.append(EarlyStopping(monitor='val_acc', mode='max', patience=6))

model.summary()

history = model.fit(X_train, y_train,batch_size=BATCH_SIZE, epochs=NB_EPOCH,verbose=VERBOSE, validation_data=(X_valid, y_valid), callbacks=callbacks_list)

with open('/content/training_history/His_Dict_without_Nom', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)


print("############## Training is Done Successfully ###################")
#model.save('DC_normal_model.hdf5')
#model=load_model('AWF.hdf5')

# Start evaluating model with testing data
score_test = model.evaluate(X_test, y_test, verbose=VERBOSE)
print("Testing closed accuracy_without_norm:", score_test[1])

model_penultimate = Model(model.input, model.layers[-2].output)
model_penultimate.save('/content/trained_models/SETA_without_softmax.hdf5')

del model


Number of Epoch:  450
Loading and preparing data for training, and evaluating the model
Data dimensions:
X: Training data's shape :  (554, 500)
y: Training data's shape :  (554,)
X: Validation data's shape :  (142, 500)
y: Validation data's shape :  (142,)
X: Testing data's shape :  (96, 500)
y: Testing data's shape :  (96,)
Building and training DF model
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1 (Conv1D)              (None, 485, 32)           544       
                                                                 
 conv2 (Conv1D)              (None, 470, 32)           16416     
                                                                 
 cov1_dropout (Dropout)      (None, 470, 32)           0         
                                                                 
 conv3 (Conv1D)              (None, 455, 32)           16416     
                          

9/9 - 13s - loss: 2.4453 - accuracy: 0.1625 - val_loss: 1.9253 - val_accuracy: 0.2188 - 13s/epoch - 1s/step
Epoch 2/450


9/9 - 0s - loss: 1.9274 - accuracy: 0.2166 - val_loss: 1.8993 - val_accuracy: 0.3646 - 86ms/epoch - 10ms/step
Epoch 3/450


9/9 - 0s - loss: 1.8958 - accuracy: 0.2635 - val_loss: 1.8888 - val_accuracy: 0.4062 - 93ms/epoch - 10ms/step
Epoch 4/450


9/9 - 0s - loss: 1.8992 - accuracy: 0.2599 - val_loss: 1.8247 - val_accuracy: 0.3542 - 81ms/epoch - 9ms/step
Epoch 5/450


...

9/9 - 0s - loss: 0.0507 - accuracy: 0.9783 - val_loss: 0.2085 - val_accuracy: 0.9792 - 99ms/epoch - 11ms/step
Epoch 449/450


9/9 - 0s - loss: 0.0307 - accuracy: 0.9874 - val_loss: 0.1775 - val_accuracy: 0.9792 - 94ms/epoch - 10ms/step
Epoch 450/450


9/9 - 0s - loss: 0.0473 - accuracy: 0.9801 - val_loss: 0.1488 - val_accuracy: 0.9896 - 94ms/epoch - 10ms/step
############## Training is Done Successfully ###################
5/5 - 0s - loss: 0.0169 - accuracy: 0.9930 - 123ms/epoch - 25ms/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Testing closed accuracy_without_norm: 0.9929577708244324


In [14]:
################## Calculating mean anchor vectors for each class ######################

y_train=np.argmax(y_train,axis=1)
y_valid=np.argmax(y_valid,axis=1)
y_test=np.argmax(y_test,axis=1)

txt_O = "Mean_{Class1:.0f}"
Means={}
for i in range(NB_CLASSES):
  Means[txt_O.format(Class1=i)]=np.array([0]*NB_CLASSES)



tflite_model_predictions = model_penultimate.predict(X_train)


print('Shape: ',tflite_model_predictions.shape)
count=[0]*NB_CLASSES

for i in range(len(tflite_model_predictions)):
  k=np.argmax(tflite_model_predictions[i])
  if (np.argmax(tflite_model_predictions[i])==y_train[i]):
    Means[txt_O.format(Class1=y_train[i])]=Means[txt_O.format(Class1=y_train[i])]+tflite_model_predictions[i]
    count[y_train[i]]+=1
#print("Counts: ",count)

Mean_Vectors=[]
for i in range(NB_CLASSES):
  Means[txt_O.format(Class1=i)]=Means[txt_O.format(Class1=i)]/count[i]
  Mean_Vectors.append(Means[txt_O.format(Class1=i)])

Mean_vectors=np.array(Mean_Vectors)
np.save('/content/temp_variables/Mean_vectors.npy', Mean_vectors, allow_pickle=True)



tflite_model_predictions=model_penultimate.predict(X_valid)

Mean_Vectors=Mean_vectors

print()


18/18 [==============================] - 0s 2ms/step
Shape:  (554, 8)
3/3 [==============================] - 0s 3ms/step



In [15]:
##################################################################################################
print()
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    #dist=np.dot(Mean_Vectors[y_valid[i]],tflite_model_predictions[i])
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Distances[txt_1.format(Class1=y_valid[i])].append(dist)

#print(Distances)
TH=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  TH[j]=Dist[int(len(Dist)*0.9)]



Threasholds_1=np.array(TH)
np.save('Threasholds_1.npy',Threasholds_1)

print("Thresholds for method 1 calculated")
print()
##################################################################################################
print()
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    #dist=np.dot(Mean_Vectors[y_valid[i]],tflite_model_predictions[i])
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(y_valid[i]):
        #Tot+=np.dot(Mean_Vectors[k],tflite_model_predictions[i])-dist
        Tot+=(np.linalg.norm(Mean_Vectors[k]-tflite_model_predictions[i])-dist)
    Distances[txt_1.format(Class1=y_valid[i])].append(Tot)

#print(Distances)
TH=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  TH[j]=Dist[int(len(Dist)*0.1)]

Threasholds_2=np.array(TH)
np.save('Threasholds_2.npy',Threasholds_2)

print("Thresholds for method 2 calculated")

print()
##################################################################################################
print()
txt_1 = "Dist_{Class1:.0f}"
Distances={}
for i in range(NB_CLASSES):
  Distances[txt_1.format(Class1=i)]=[]

for i in range(len(tflite_model_predictions)):
  if (y_valid[i]==np.argmax(tflite_model_predictions[i])):
    #dist=np.dot(Mean_Vectors[y_valid[i]],tflite_model_predictions[i])
    dist = np.linalg.norm(Mean_Vectors[y_valid[i]]-tflite_model_predictions[i])
    Tot=0
    for k in range(NB_CLASSES):
      if k!=int(y_valid[i]):
        #Tot=np.dot(Mean_Vectors[k],tflite_model_predictions[i])-dist
        Tot+=np.linalg.norm(Mean_Vectors[k]-tflite_model_predictions[i])
    Tot=dist/Tot
    Distances[txt_1.format(Class1=y_valid[i])].append(Tot)

#print(Distances)
TH=[0]*NB_CLASSES
for j in range(NB_CLASSES):
  Distances[txt_1.format(Class1=j)].sort()
  Dist=Distances[txt_1.format(Class1=j)]
  TH[j]=Dist[int(len(Dist)*0.9)]

Threasholds_3=np.array(TH)
np.save('Threasholds_3.npy',Threasholds_3)


print("Thresholds for method 3 calculated")


Thresholds for method 1 calculated


Thresholds for method 2 calculated


Thresholds for method 3 calculated


In [16]:
############################################################################################

######################## Evaluating results ##########################
dataset_dir="/content/dataset/"
X_open = np.load(dataset_dir+'X_open.npy')
y__open = np.load(dataset_dir+'y_open.npy')
y_open=np.array([NB_CLASSES]*len(y__open))

X_open = X_open.astype('float32')
y_open = y_open.astype('int8')

X_open = X_open[:, :,np.newaxis]

tflite_model_predictions = model_penultimate.predict(X_test)
tflite_model_predictions_open = model_penultimate.predict(X_open)

#############################################################################################################
print()
print("############## Distance Method 1 #################################")
print()
prediction_classes = []
for i in range(len(tflite_model_predictions)):

    d = np.argmax(tflite_model_predictions[i], axis=0)
    if np.linalg.norm(tflite_model_predictions[i] - Mean_vectors[d]) > Threasholds_1[d]:
        prediction_classes.append(NB_CLASSES)

    else:
        prediction_classes.append(d)

prediction_classes = np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test)
print('Test accuracy Normal model_Closed_set :', acc_Close)


prediction_classes_open = []
for i in range(len(tflite_model_predictions_open)):

    d = np.argmax(tflite_model_predictions_open[i], axis=0)
    if np.linalg.norm(tflite_model_predictions_open[i] - Mean_vectors[d]) > Threasholds_1[d]:
        prediction_classes_open.append(NB_CLASSES)

    else:
        prediction_classes_open.append(d)

prediction_classes_open = np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open)
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix = []
for i in range(NB_CLASSES + 1):
    Matrix.append(np.zeros(NB_CLASSES + 1))

for i in range(len(y_test)):
    Matrix[y_test[i]][prediction_classes[i]] += 1

for i in range(len(y_open)):
    Matrix[y_open[i]][prediction_classes_open[i]] += 1

#print(Matrix)

print()
F1_Score = New_F1_Score(Matrix)
print("Average novel F1_Score: ", F1_Score)

print()
F1_Score = Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
F1_Score = Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)

#############################################################################################################
print()
print("############## Distance Method 2 #################################")
print()
prediction_classes = []
for i in range(len(tflite_model_predictions)):
    d = np.argmax(tflite_model_predictions[i], axis=0)
    dist = np.linalg.norm(Mean_vectors[d] - tflite_model_predictions[i])
    Tot = 0
    for k in range(NB_CLASSES):
        if k != d:
            Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions[i]) - dist

    if Tot < Threasholds_2[d]:
        prediction_classes.append(NB_CLASSES)

    else:
        prediction_classes.append(d)

prediction_classes = np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test)
print('Test accuracy Normal model_Closed_set :', acc_Close)

prediction_classes_open = []
for i in range(len(tflite_model_predictions_open)):
    d = np.argmax(tflite_model_predictions_open[i], axis=0)
    dist = np.linalg.norm(Mean_vectors[d] - tflite_model_predictions_open[i])
    Tot = 0
    for k in range(NB_CLASSES):
        if k != d:
            Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions_open[i]) - dist

    if Tot < Threasholds_2[d]:
        prediction_classes_open.append(NB_CLASSES)

    else:
        prediction_classes_open.append(d)

prediction_classes_open = np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open)
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix = []
for i in range(NB_CLASSES + 1):
    Matrix.append(np.zeros(NB_CLASSES + 1))

for i in range(len(y_test)):
    Matrix[y_test[i]][prediction_classes[i]] += 1

for i in range(len(y_open)):
    Matrix[y_open[i]][prediction_classes_open[i]] += 1

#print(Matrix)

print()
F1_Score = New_F1_Score(Matrix)
print("Average novel F1_Score: ", F1_Score)

print()
F1_Score = Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
F1_Score = Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)

#############################################################################################################
print()
print("############## Distance Method 3 #################################")
print()
prediction_classes = []
for i in range(len(tflite_model_predictions)):
    d = np.argmax(tflite_model_predictions[i], axis=0)
    dist = np.linalg.norm(Mean_vectors[d] - tflite_model_predictions[i])
    Tot = 0
    for k in range(NB_CLASSES):
        if k != d:
            Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions[i])

    Tot = dist / Tot
    if Tot > Threasholds_3[d]:
        prediction_classes.append(NB_CLASSES)

    else:
        prediction_classes.append(d)

prediction_classes = np.array(prediction_classes)
acc_Close = accuracy_score(prediction_classes, y_test)
print('Test accuracy Normal model_Closed_set :', acc_Close)

prediction_classes_open = []
for i in range(len(tflite_model_predictions_open)):
    d = np.argmax(tflite_model_predictions_open[i], axis=0)
    dist = np.linalg.norm(Mean_vectors[d] - tflite_model_predictions_open[i])
    Tot = 0
    for k in range(NB_CLASSES):
        if k != d:
            Tot += np.linalg.norm(Mean_vectors[k] - tflite_model_predictions_open[i])

    Tot = dist / Tot
    if Tot > Threasholds_3[d]:
        prediction_classes_open.append(NB_CLASSES)

    else:
        prediction_classes_open.append(d)

prediction_classes_open = np.array(prediction_classes_open)

acc_Open = accuracy_score(prediction_classes_open, y_open)
print('Test accuracy Normal model_Open_set :', acc_Open)

Matrix = []
for i in range(NB_CLASSES + 1):
    Matrix.append(np.zeros(NB_CLASSES + 1))

for i in range(len(y_test)):
    Matrix[y_test[i]][prediction_classes[i]] += 1

for i in range(len(y_open)):
    Matrix[y_open[i]][prediction_classes_open[i]] += 1

#print(Matrix)

print()
F1_Score = New_F1_Score(Matrix)
print("Average novel F1_Score: ", F1_Score)

print()
F1_Score = Micro_F1(Matrix)
print("Average Micro F1_Score: ", F1_Score)

print()
F1_Score = Macro_F1(Matrix)
print("Average Macro F1_Score: ", F1_Score)

38/38 [==============================] - 0s 2ms/step

############## Distance Method 1 #################################

Test accuracy Normal model_Closed_set : 0.8380281690140845
Test accuracy Normal model_Open_set : 0.8400673400673401

Precision:  0.36644049548012964
Recall:  0.8946759259259259
Average novel F1_Score:  0.5199289794756152

Micro Precision:  0.38387096774193546
Micro Recall:  0.8380281690140845
Average Micro F1_Score:  0.5265486682571071

Precision: 0.5768641948329448
Recall: 0.8382352941176471
Average Macro F1_Score:  0.683411913268745

############## Distance Method 2 #################################

Test accuracy Normal model_Closed_set : 0.823943661971831
Test accuracy Normal model_Open_set : 0.8905723905723906

Precision:  0.4794878156521398
Recall:  0.866510372264848
Average novel F1_Score:  0.6173576848470915

Micro Precision:  0.47368421052631576
Micro Recall:  0.823943661971831
Average Micro F1_Score:  0.601542411816734

Precision: 0.7037378008694193
Recall